In [1]:
import csv
import json
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict
np.random.seed(90501)

In [2]:
from tensorflow.keras.utils import to_categorical

# train set
train_set = pd.read_csv('../data/final/train_reconstructed.csv')
X_train = train_set.iloc[:,:-1].values
y_train = train_set['edge'].values

# dev set
test_set = pd.read_csv('../data/final/dev-test.csv')
X_test = test_set.iloc[:,:-1].values
y_test = test_set['edge'].values

# X_train = X_train.reshape(-1, X_train.shape[1], 1)
# X_test = X_test.reshape(-1, X_test.shape[1], 1)

# y_train = to_categorical(y_train, 2)
# y_test = to_categorical(y_test, 2)

# X_flatten_train = [instance.flatten() for instance in X_train]
# X_flatten_test = [instance.flatten() for instance in X_test]

Just using one layer and seeing what happens gives us a good idea of what the simplest model could give us in terms of accuracy (around 72% - 78%)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD

nnmodel_1 = Sequential()
nnmodel_1.add(Flatten(input_shape= X_train.shape[1:]))
nnmodel_1.add(Dense(1,activation='sigmoid'))

nnmodel_1.compile(loss='binary_crossentropy', optimizer = SGD(lr=0.01, decay =2e-6, momentum =0.9, nesterov = True),
                 metrics = ['AUC'])

nnmodel_1.summary()
nnmodel_1.fit(X_train, y_train, epochs = 20, batch_size = 128,verbose=2)

score = nnmodel_1.evaluate(X_test, y_test, batch_size = 128)

print("\nOn the test set: the loss is {} and the AUC is {}".format(score[0], score[1]))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
744/744 - 1s - loss: 0.2019 - auc: 0.9774
Epoch 2/20
744/744 - 1s - loss: 0.1528 - auc: 0.9836
Epoch 3/20
744/744 - 1s - loss: 0.1517 - auc: 0.9837
Epoch 4/20
744/744 - 1s - loss: 0.1504 - auc: 0.9841
Epoch 5/20
744/744 - 1s - loss: 0.1484 - auc: 0.9848
Epoch 6/20
744/744 - 1s - loss: 0.1485 - auc: 0.9849
Epoch 7/20
744/744 - 1s - loss: 0.1472 - auc: 0.9852
Epoch 8/20
744/744 - 1s - loss: 0.1464 - auc: 0.9855
Epoch 9/20
744/744 - 1s - loss: 0.1464 - auc: 0.9855
Epoch 10/20
744/744 - 1s - loss: 0.1454 - a

We now set a more complex model testing these activation functions: **['relu', 'tanh', 'sigmoid', 'selu', 'elu']**. Using two layers of 11 neurons each. We get that usually the best activation function is **tanh** with nearly **78% - 80%** of the accuracy in the test set.

In [7]:
import kerastuner as kt
from tensorflow import keras
from kerastuner.engine.hyperparameters import HyperParameters

def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 1, 10)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=2,
                                            max_value=16,
                                            step=2),
                                            activation= hp.Choice('activation_'+str(i), ['relu','tanh','sigmoid','selu','elu'])))
        model.add(Dropout(hp.Float('dropout_' +str(i),
                                                min_value=0.0,
                                                max_value=0.5,
                                                step=0.1)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(
                    loss='binary_crossentropy', 
                    optimizer = keras.optimizers.Adam(
                                    hp.Choice('learning_rate', 
                                                [1e-2, 1e-3, 1e-4])),
                    metrics = ['AUC']
                    )
    return model



In [8]:
from kerastuner.tuners import BayesianOptimization
tuner = BayesianOptimization(
    build_model,
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=50,
    directory='./',
    project_name='test1')


In [9]:
tuner.search(x=X_train,y=y_train,
             epochs=20,
             verbose = 2,
             validation_data=(X_test, y_test))

Epoch 1/20
2974/2974 - 4s - loss: 0.4007 - auc: 0.8777 - val_loss: 0.5113 - val_auc: 0.8216
Epoch 2/20
2974/2974 - 3s - loss: 0.2797 - auc: 0.9406 - val_loss: 0.5571 - val_auc: 0.8296
Epoch 3/20
2974/2974 - 4s - loss: 0.2410 - auc: 0.9541 - val_loss: 0.5466 - val_auc: 0.8276
Epoch 4/20
2974/2974 - 4s - loss: 0.2184 - auc: 0.9613 - val_loss: 0.5736 - val_auc: 0.8314
Epoch 5/20
2974/2974 - 4s - loss: 0.2015 - auc: 0.9698 - val_loss: 0.6298 - val_auc: 0.8388
Epoch 6/20
2974/2974 - 4s - loss: 0.1926 - auc: 0.9725 - val_loss: 0.6582 - val_auc: 0.8439
Epoch 7/20
2974/2974 - 4s - loss: 0.1924 - auc: 0.9728 - val_loss: 0.5594 - val_auc: 0.8427
Epoch 8/20
2974/2974 - 4s - loss: 0.1909 - auc: 0.9726 - val_loss: 0.6075 - val_auc: 0.8332
Epoch 9/20
2974/2974 - 4s - loss: 0.1879 - auc: 0.9737 - val_loss: 0.7341 - val_auc: 0.8368
Epoch 10/20
2974/2974 - 5s - loss: 0.1876 - auc: 0.9736 - val_loss: 0.6950 - val_auc: 0.8394
Epoch 11/20
2974/2974 - 5s - loss: 0.1891 - auc: 0.9732 - val_loss: 0.5977 - va

Epoch 1/20
2974/2974 - 7s - loss: 0.5247 - auc: 0.8146 - val_loss: 0.4664 - val_auc: 0.8582
Epoch 2/20
2974/2974 - 6s - loss: 0.4608 - auc: 0.8519 - val_loss: 0.4613 - val_auc: 0.8454
Epoch 3/20
2974/2974 - 6s - loss: 0.4492 - auc: 0.8555 - val_loss: 0.4727 - val_auc: 0.8387
Epoch 4/20
2974/2974 - 6s - loss: 0.4446 - auc: 0.8570 - val_loss: 0.4642 - val_auc: 0.8452
Epoch 5/20
2974/2974 - 5s - loss: 0.4450 - auc: 0.8566 - val_loss: 0.4893 - val_auc: 0.8388
Epoch 6/20
2974/2974 - 4s - loss: 0.4403 - auc: 0.8601 - val_loss: 0.4473 - val_auc: 0.8468
Epoch 7/20
2974/2974 - 5s - loss: 0.4386 - auc: 0.8597 - val_loss: 0.4626 - val_auc: 0.8518
Epoch 8/20
2974/2974 - 4s - loss: 0.4415 - auc: 0.8579 - val_loss: 0.4888 - val_auc: 0.8479
Epoch 9/20
2974/2974 - 5s - loss: 0.4404 - auc: 0.8585 - val_loss: 0.4450 - val_auc: 0.8594
Epoch 10/20
2974/2974 - 5s - loss: 0.4394 - auc: 0.8586 - val_loss: 0.5055 - val_auc: 0.8259
Epoch 11/20
2974/2974 - 4s - loss: 0.4335 - auc: 0.8626 - val_loss: 0.4728 - va

Epoch 1/20
2974/2974 - 4s - loss: 0.1545 - auc: 0.9811 - val_loss: 0.8362 - val_auc: 0.8399
Epoch 2/20
2974/2974 - 3s - loss: 0.1390 - auc: 0.9838 - val_loss: 0.7423 - val_auc: 0.8274
Epoch 3/20
2974/2974 - 3s - loss: 0.1378 - auc: 0.9843 - val_loss: 0.6850 - val_auc: 0.8346
Epoch 4/20
2974/2974 - 3s - loss: 0.1364 - auc: 0.9853 - val_loss: 0.6188 - val_auc: 0.8406
Epoch 5/20
2974/2974 - 3s - loss: 0.1345 - auc: 0.9858 - val_loss: 0.7094 - val_auc: 0.8320
Epoch 6/20
2974/2974 - 3s - loss: 0.1339 - auc: 0.9857 - val_loss: 0.7573 - val_auc: 0.8344
Epoch 7/20
2974/2974 - 3s - loss: 0.1328 - auc: 0.9858 - val_loss: 0.6926 - val_auc: 0.8378
Epoch 8/20
2974/2974 - 3s - loss: 0.1328 - auc: 0.9859 - val_loss: 0.6540 - val_auc: 0.8442
Epoch 9/20
2974/2974 - 3s - loss: 0.1324 - auc: 0.9860 - val_loss: 0.6765 - val_auc: 0.8460
Epoch 10/20
2974/2974 - 3s - loss: 0.1317 - auc: 0.9860 - val_loss: 0.7242 - val_auc: 0.8427
Epoch 11/20
2974/2974 - 3s - loss: 0.1305 - auc: 0.9864 - val_loss: 0.7434 - va

Epoch 1/20
2974/2974 - 5s - loss: 0.6944 - auc: 0.5029 - val_loss: 0.6932 - val_auc: 0.5000
Epoch 2/20
2974/2974 - 5s - loss: 0.3886 - auc: 0.9004 - val_loss: 0.5205 - val_auc: 0.8476
Epoch 3/20
2974/2974 - 5s - loss: 0.2680 - auc: 0.9545 - val_loss: 0.5144 - val_auc: 0.8445
Epoch 4/20
2974/2974 - 4s - loss: 0.2566 - auc: 0.9573 - val_loss: 0.5018 - val_auc: 0.8463
Epoch 5/20
2974/2974 - 5s - loss: 0.2507 - auc: 0.9586 - val_loss: 0.4904 - val_auc: 0.8400
Epoch 6/20
2974/2974 - 5s - loss: 0.2472 - auc: 0.9593 - val_loss: 0.5548 - val_auc: 0.8379
Epoch 7/20
2974/2974 - 4s - loss: 0.2447 - auc: 0.9599 - val_loss: 0.5093 - val_auc: 0.8347
Epoch 8/20
2974/2974 - 4s - loss: 0.2419 - auc: 0.9605 - val_loss: 0.4951 - val_auc: 0.8355
Epoch 9/20
2974/2974 - 5s - loss: 0.2422 - auc: 0.9607 - val_loss: 0.5103 - val_auc: 0.8434
Epoch 10/20
2974/2974 - 4s - loss: 0.2399 - auc: 0.9614 - val_loss: 0.5296 - val_auc: 0.8313
Epoch 11/20
2974/2974 - 4s - loss: 0.2392 - auc: 0.9616 - val_loss: 0.5488 - va

Epoch 1/20
2974/2974 - 4s - loss: 0.2716 - auc: 0.9564 - val_loss: 0.6318 - val_auc: 0.8344
Epoch 2/20
2974/2974 - 4s - loss: 0.1692 - auc: 0.9749 - val_loss: 0.6582 - val_auc: 0.8373
Epoch 3/20
2974/2974 - 4s - loss: 0.1610 - auc: 0.9776 - val_loss: 0.6668 - val_auc: 0.8392
Epoch 4/20
2974/2974 - 4s - loss: 0.1550 - auc: 0.9799 - val_loss: 0.7334 - val_auc: 0.8381
Epoch 5/20
2974/2974 - 4s - loss: 0.1506 - auc: 0.9813 - val_loss: 0.6740 - val_auc: 0.8344
Epoch 6/20
2974/2974 - 4s - loss: 0.1504 - auc: 0.9814 - val_loss: 0.7235 - val_auc: 0.8402
Epoch 7/20
2974/2974 - 4s - loss: 0.1484 - auc: 0.9825 - val_loss: 0.6543 - val_auc: 0.8368
Epoch 8/20
2974/2974 - 4s - loss: 0.1474 - auc: 0.9826 - val_loss: 0.8082 - val_auc: 0.8407
Epoch 9/20
2974/2974 - 4s - loss: 0.1468 - auc: 0.9829 - val_loss: 0.7514 - val_auc: 0.8426
Epoch 10/20
2974/2974 - 4s - loss: 0.1454 - auc: 0.9835 - val_loss: 0.7538 - val_auc: 0.8423
Epoch 11/20
2974/2974 - 5s - loss: 0.1455 - auc: 0.9836 - val_loss: 0.6347 - va

...

Epoch 1/20
2974/2974 - 3s - loss: 1.5635 - auc: 0.6330 - val_loss: 0.7140 - val_auc: 0.6900
Epoch 2/20
2974/2974 - 3s - loss: 0.3776 - auc: 0.9178 - val_loss: 0.6244 - val_auc: 0.7978
Epoch 3/20
2974/2974 - 3s - loss: 0.2259 - auc: 0.9672 - val_loss: 0.6282 - val_auc: 0.8349
Epoch 4/20
2974/2974 - 3s - loss: 0.1750 - auc: 0.9792 - val_loss: 0.6492 - val_auc: 0.8519
Epoch 5/20
2974/2974 - 3s - loss: 0.1536 - auc: 0.9835 - val_loss: 0.6320 - val_auc: 0.8550
Epoch 6/20
2974/2974 - 3s - loss: 0.1457 - auc: 0.9848 - val_loss: 0.6466 - val_auc: 0.8567
Epoch 7/20
2974/2974 - 3s - loss: 0.1393 - auc: 0.9859 - val_loss: 0.6200 - val_auc: 0.8570
Epoch 8/20
2974/2974 - 3s - loss: 0.1350 - auc: 0.9864 - val_loss: 0.6158 - val_auc: 0.8559
Epoch 9/20
2974/2974 - 3s - loss: 0.1320 - auc: 0.9869 - val_loss: 0.6228 - val_auc: 0.8558
Epoch 10/20
2974/2974 - 3s - loss: 0.1306 - auc: 0.9871 - val_loss: 0.6511 - val_auc: 0.8567
Epoch 11/20
2974/2974 - 3s - loss: 0.1289 - auc: 0.9874 - val_loss: 0.6641 - va

Epoch 1/20
2974/2974 - 3s - loss: 0.9430 - auc: 0.8116 - val_loss: 0.6661 - val_auc: 0.7700
Epoch 2/20
2974/2974 - 3s - loss: 0.2638 - auc: 0.9563 - val_loss: 0.6453 - val_auc: 0.8375
Epoch 3/20
2974/2974 - 3s - loss: 0.1841 - auc: 0.9770 - val_loss: 0.6284 - val_auc: 0.8493
Epoch 4/20
2974/2974 - 3s - loss: 0.1577 - auc: 0.9825 - val_loss: 0.6329 - val_auc: 0.8525
Epoch 5/20
2974/2974 - 3s - loss: 0.1460 - auc: 0.9848 - val_loss: 0.6165 - val_auc: 0.8536
Epoch 6/20
2974/2974 - 3s - loss: 0.1389 - auc: 0.9856 - val_loss: 0.6341 - val_auc: 0.8545
Epoch 7/20
2974/2974 - 3s - loss: 0.1342 - auc: 0.9864 - val_loss: 0.6533 - val_auc: 0.8547
Epoch 8/20
2974/2974 - 3s - loss: 0.1312 - auc: 0.9867 - val_loss: 0.6504 - val_auc: 0.8551
Epoch 9/20
2974/2974 - 3s - loss: 0.1287 - auc: 0.9872 - val_loss: 0.6870 - val_auc: 0.8558
Epoch 10/20
2974/2974 - 3s - loss: 0.1271 - auc: 0.9875 - val_loss: 0.6449 - val_auc: 0.8533
Epoch 11/20
2974/2974 - 3s - loss: 0.1259 - auc: 0.9876 - val_loss: 0.6562 - va

Epoch 1/20
2974/2974 - 3s - loss: 0.2301 - auc: 0.9713 - val_loss: 0.5838 - val_auc: 0.8531
Epoch 2/20
2974/2974 - 3s - loss: 0.1220 - auc: 0.9886 - val_loss: 0.6033 - val_auc: 0.8503
Epoch 3/20
2974/2974 - 3s - loss: 0.1179 - auc: 0.9895 - val_loss: 0.6368 - val_auc: 0.8451
Epoch 4/20
2974/2974 - 3s - loss: 0.1162 - auc: 0.9898 - val_loss: 0.6812 - val_auc: 0.8459
Epoch 5/20
2974/2974 - 3s - loss: 0.1152 - auc: 0.9900 - val_loss: 0.6909 - val_auc: 0.8447
Epoch 6/20
2974/2974 - 3s - loss: 0.1143 - auc: 0.9902 - val_loss: 0.6950 - val_auc: 0.8425
Epoch 7/20
2974/2974 - 3s - loss: 0.1135 - auc: 0.9903 - val_loss: 0.6666 - val_auc: 0.8420
Epoch 8/20
2974/2974 - 3s - loss: 0.1125 - auc: 0.9905 - val_loss: 0.7029 - val_auc: 0.8414
Epoch 9/20
2974/2974 - 3s - loss: 0.1116 - auc: 0.9906 - val_loss: 0.7194 - val_auc: 0.8460
Epoch 10/20
2974/2974 - 3s - loss: 0.1106 - auc: 0.9908 - val_loss: 0.7040 - val_auc: 0.8476
Epoch 11/20
2974/2974 - 3s - loss: 0.1101 - auc: 0.9909 - val_loss: 0.7432 - va

Epoch 1/20
2974/2974 - 4s - loss: 0.9961 - auc: 0.7705 - val_loss: 0.6720 - val_auc: 0.6799
Epoch 2/20
2974/2974 - 3s - loss: 0.4249 - auc: 0.9022 - val_loss: 0.6376 - val_auc: 0.7471
Epoch 3/20
2974/2974 - 3s - loss: 0.3073 - auc: 0.9480 - val_loss: 0.6082 - val_auc: 0.7938
Epoch 4/20
2974/2974 - 4s - loss: 0.2378 - auc: 0.9680 - val_loss: 0.6125 - val_auc: 0.8160
Epoch 5/20
2974/2974 - 4s - loss: 0.1972 - auc: 0.9763 - val_loss: 0.6268 - val_auc: 0.8274
Epoch 6/20
2974/2974 - 3s - loss: 0.1730 - auc: 0.9808 - val_loss: 0.6427 - val_auc: 0.8346
Epoch 7/20
2974/2974 - 3s - loss: 0.1617 - auc: 0.9826 - val_loss: 0.6398 - val_auc: 0.8387
Epoch 8/20
2974/2974 - 3s - loss: 0.1550 - auc: 0.9838 - val_loss: 0.6664 - val_auc: 0.8405
Epoch 9/20
2974/2974 - 3s - loss: 0.1494 - auc: 0.9847 - val_loss: 0.6521 - val_auc: 0.8426
Epoch 10/20
2974/2974 - 3s - loss: 0.1456 - auc: 0.9851 - val_loss: 0.6694 - val_auc: 0.8440
Epoch 11/20
2974/2974 - 3s - loss: 0.1428 - auc: 0.9855 - val_loss: 0.6717 - va

INFO:tensorflow:Oracle triggered exit


In [114]:
best_hp = tuner.get_best_hyperparameters()[0]
print(best_hp.values)
model = tuner.get_best_models(num_models=1)[0]
score = model.evaluate(X_test, y_test,verbose=2)
print("On the test set: the loss is {} and the AUC is {}".format(score[0], score[1]))

{'num_layers': 6, 'units_0': 6, 'activation_0': 'elu', 'dropout_0': 0.4, 'learning_rate': 0.001, 'units_1': 12, 'activation_1': 'sigmoid', 'dropout_1': 0.4, 'units_2': 4, 'activation_2': 'sigmoid', 'dropout_2': 0.5, 'units_3': 16, 'activation_3': 'sigmoid', 'dropout_3': 0.4, 'units_4': 8, 'activation_4': 'relu', 'dropout_4': 0.4, 'units_5': 4, 'activation_5': 'elu', 'dropout_5': 0.2, 'units_6': 6, 'activation_6': 'selu', 'dropout_6': 0.5, 'units_7': 12, 'activation_7': 'selu', 'dropout_7': 0.4, 'units_8': 14, 'activation_8': 'relu', 'dropout_8': 0.4}
153/153 - 0s - loss: 0.4937 - auc: 0.8710
On the test set: the loss is 0.4936865568161011 and the AUC is 0.8710033297538757


In [122]:
for i in model.layers[0:13:2]:
    print(i.units,i.activation)

6 <function elu at 0x000001FCBB44B378>
12 <function sigmoid at 0x000001FCBB44B730>
4 <function sigmoid at 0x000001FCBB44B730>
16 <function sigmoid at 0x000001FCBB44B730>
8 <function relu at 0x000001FCBB44B620>
4 <function elu at 0x000001FCBB44B378>
1 <function sigmoid at 0x000001FCBB44B730>


In [96]:
for i in model.layers[1:12:2]:
    print(i.rate)

0.4
0.4
0.5
0.4
0.4
0.2


In [123]:
model_1 = Sequential()

model_1.add(Dense(6,activation='elu'))
model_1.add(Dropout(0.4))
model_1.add(Dense(12,activation='sigmoid'))
model_1.add(Dropout(0.4))
model_1.add(Dense(4,activation='sigmoid'))
model_1.add(Dropout(0.5))
model_1.add(Dense(16,activation='sigmoid'))
model_1.add(Dropout(0.4))
model_1.add(Dense(8,activation='relu'))
model_1.add(Dropout(0.4))
model_1.add(Dense(4,activation='elu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1,activation='sigmoid'))

model_1.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.001),
                 metrics = ['AUC'])

model_1.fit(X_train, y_train, epochs = 20, batch_size = 32,verbose=2)

score = model_1.evaluate(X_test, y_test, batch_size = 32)

print("On the test set: the loss is {} and the AUC is {}".format(score[0], score[1]))

Epoch 1/20
2974/2974 - 4s - loss: 0.6864 - auc: 0.5363
Epoch 2/20
2974/2974 - 4s - loss: 0.4149 - auc: 0.8923
Epoch 3/20
2974/2974 - 4s - loss: 0.3522 - auc: 0.9209
Epoch 4/20
2974/2974 - 4s - loss: 0.3356 - auc: 0.9249
Epoch 5/20
2974/2974 - 5s - loss: 0.3269 - auc: 0.9267
Epoch 6/20
2974/2974 - 7s - loss: 0.3272 - auc: 0.9264
Epoch 7/20
2974/2974 - 4s - loss: 0.3293 - auc: 0.9253
Epoch 8/20
2974/2974 - 4s - loss: 0.3260 - auc: 0.9262
Epoch 9/20
2974/2974 - 4s - loss: 0.3269 - auc: 0.9263
Epoch 10/20
2974/2974 - 4s - loss: 0.3253 - auc: 0.9268
Epoch 11/20
2974/2974 - 4s - loss: 0.3226 - auc: 0.9275
Epoch 12/20
2974/2974 - 4s - loss: 0.3249 - auc: 0.9263
Epoch 13/20
2974/2974 - 4s - loss: 0.3265 - auc: 0.9259
Epoch 14/20
2974/2974 - 4s - loss: 0.3243 - auc: 0.9269
Epoch 15/20
2974/2974 - 4s - loss: 0.3256 - auc: 0.9266
Epoch 16/20
2974/2974 - 4s - loss: 0.3227 - auc: 0.9272
Epoch 17/20
2974/2974 - 3s - loss: 0.3233 - auc: 0.9262
Epoch 18/20
2974/2974 - 3s - loss: 0.3195 - auc: 0.9289
E

After runing the test againg, as stated previously, this model does not now an improvement against nnmodel_2 and that is the best NN model explored at the moment.

# Make prediction with full dataset

In [24]:
from tensorflow.keras.utils import to_categorical

# train set
train_set = pd.read_csv('../data/final/train_reconstructed.csv')
X_train = train_set.iloc[:,:-1].values
y_train = train_set['edge'].values

# dev set
test_set = pd.read_csv('../data/final/dev-test.csv')
X_test = test_set.iloc[:,:-1].values
y_test = test_set['edge'].values

X_big = np.concatenate((X_train,X_test),0)
y_big = np.concatenate((y_train,y_test),0)


# #X_big = X_big.reshape(-1, X_big.shape[1], 1)
# y_big = to_categorical(y_big, 2)
# X_flatten_big = [instance.flatten() for instance in X_big]

In [25]:
# Training full model
test_final = pd.read_csv('../data/final/test-final.csv')
test_final = test_final.values

In [30]:
model.predict(test_final)[:,0]

array([0.1047942 , 0.9808475 , 0.11504483, ..., 0.10500063, 0.11209269,
       0.98084897], dtype=float32)

In [31]:
pred = model.predict(test_final)[:,0]
submission = {
    'Id': range(1,len(pred)+1),
    'Predicted': pred
}

submission_df = pd.DataFrame(data=submission)
submission_df.to_csv('../data/final/sub.csv', index=False)